# Function Calling Demo: Current Weather

Evolved version of the OpenAI function calling example found at https://platform.openai.com/docs/guides/function-calling

### Setup
Load the API keys and relevant Python libaries. Create an OpenAI client.

In [1]:
from geopy.geocoders import Nominatim
import requests, json, dotenv
from openai import OpenAI

openweather_api_key = (dotenv.dotenv_values()).get("OPENWEATHERMAP_API_KEY")
openai_api_key = (dotenv.dotenv_values()).get("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)
base_url = "https://api.openweathermap.org/data/2.5/weather?"
question = "What's the weather like in San Francisco, Tokyo, Paris, and Cologne in Germany?"

### Function to fetch the current weather for a given location

Geolocation information is retrieved using GeoPy: https://github.com/geopy/geopy
Weather information is retrieved via https://openweathermap.org/. Use of the API requires a subscription. A free tier is available.

In [2]:
def get_current_weather(location):
    loc = Nominatim(user_agent="Get Location")
    getLoc = loc.geocode(location)
    locname = getLoc.address
    complete_url = base_url + "lat=" + str(getLoc.latitude) + "&lon=" + str(getLoc.longitude) + "&appid=" + openweather_api_key
    response = requests.get(complete_url)
    result = response.json()
    if result["cod"] != "404":
        current_temperature = result["main"]["temp"] - 273.15
        weather_description = result["weather"][0]["description"]
        return json.dumps({"location": locname, "temperature": str(current_temperature) + " °C", "weather": weather_description})
    else:
        return json.dumps({"location": location, "weather": "unknown"})

In [3]:
print(get_current_weather("Cologne, Germany"))

{"location": "K\u00f6ln, Nordrhein-Westfalen, Deutschland", "temperature": "18.879999999999995 \u00b0C", "weather": "broken clouds"}


### Interact with GPT model to get weather information for specified locations

In [4]:
def run_conversation():
    messages = [{"role": "user", "content": question}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "get_current_weather": get_current_weather,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(f"Function name: {function_name}, arguments: {function_args}")
            function_response = function_to_call(
                location=function_args.get("location"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )
        print(f"----------------------------------------\n\n")
        return second_response

### Let's try it out...

In [5]:
question = "What's the weather like in Las Vegas, Tokyo, Paris, and Cologne in Germany?"
answer=run_conversation()
print(answer.choices[0].message.content)

Function name: get_current_weather, arguments: {'location': 'Las Vegas'}
Function name: get_current_weather, arguments: {'location': 'Tokyo'}
Function name: get_current_weather, arguments: {'location': 'Paris'}
Function name: get_current_weather, arguments: {'location': 'Cologne, Germany'}
----------------------------------------


The current weather in Las Vegas is clear sky with a temperature of 23.1°C. In Tokyo, the weather is few clouds with a temperature of 20.6°C. Paris has broken clouds with a temperature of 15.9°C. Cologne in Germany also has broken clouds with a temperature of 18.9°C.


In [6]:
question = "Is the sun currently shining in Cologne, Germany?"
answer=run_conversation()
print(answer.choices[0].message.content)

Function name: get_current_weather, arguments: {'location': 'Cologne, Germany'}
----------------------------------------


The sun is currently not shining in Cologne, Germany. The weather is cloudy with broken clouds and the temperature is around 18.9°C.


In [7]:
question = "Does the sun shine in Germany currently?"
answer=run_conversation()
print(answer.choices[0].message.content)

Function name: get_current_weather, arguments: {'location': 'Germany'}
----------------------------------------


Currently, the weather in Germany is overcast with a temperature of approximately 20.56°C. Therefore, the sun might not be shining at the moment.


In [8]:
print(get_current_weather("Germany"))

{"location": "Deutschland", "temperature": "20.560000000000002 \u00b0C", "weather": "overcast clouds"}


In [9]:
question = "Does the sun shine in Las Vegas currently?"
answer=run_conversation()
print(answer.choices[0].message.content)

Function name: get_current_weather, arguments: {'location': 'Las Vegas, NV'}
----------------------------------------


Yes, the sun is currently shining in Las Vegas. The weather is clear.
